In [1]:
MAX_NB_WORDS = 40000 # max no. of words for tokenizer
MAX_SEQUENCE_LENGTH = 30 # max length of text (words) including padding
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 200 # embedding dimensions for word vectors (word2vec/GloVe)
NUM_LABELS = 3
print("[i] Loaded Parameters:\n",
      MAX_NB_WORDS,MAX_SEQUENCE_LENGTH+5,
      VALIDATION_SPLIT,EMBEDDING_DIM)

[i] Loaded Parameters:
 40000 35 0.2 200


In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [3]:
print("[i] Importing Modules...")
import numpy as np
import pandas as pd
import re, sys, os, csv, keras, pickle

from keras import regularizers, initializers, optimizers, callbacks
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding, Add, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
print("[+] Using Keras version",keras.__version__)
print("[+] Finished Importing Modules")

[i] Importing Modules...


Using TensorFlow backend.


[+] Using Keras version 2.2.4
[+] Finished Importing Modules


In [4]:
texts, labels = [], []
print("[i] Reading from csv file...", end="")
readCSV = pd.read_csv(open('training_data2.csv', encoding='utf-8'))
#null_list = readCSV[readCSV['text'].isnull()].index.tolist()
#readCSV = csv.reader("training_data2.csv", delimiter=',')
for i in range(len(readCSV)):
    texts.append(readCSV["text"][i])
    labels.append(readCSV["label"][i])

texts = list(filter(None, texts))
print("Done!")

[i] Reading from csv file...Done!


In [5]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("[i] Saved word tokenizer to file: tokenizer.pickle")

[i] Saved word tokenizer to file: tokenizer.pickle


In [6]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [7]:
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('[i] Found %s unique tokens.' % len(word_index))
data_int = pad_sequences(sequences, padding='pre', maxlen=(MAX_SEQUENCE_LENGTH-5))
data = pad_sequences(data_int, padding='post', maxlen=(MAX_SEQUENCE_LENGTH))

[i] Found 97 unique tokens.


In [8]:
labels = to_categorical(np.asarray(labels)) # convert to one-hot encoding vectors
print('[+] Shape of data tensor:', data.shape)
print('[+] Shape of label tensor:', labels.shape)

[+] Shape of data tensor: (70, 30)
[+] Shape of label tensor: (70, 3)


In [9]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.3, random_state=42)

print('[i] Number of entries in each category:')
print("[+] Training:\n",y_train.sum(axis=0))
print("[+] Validation:\n",y_val.sum(axis=0))

[i] Number of entries in each category:
[+] Training:
 [15. 18. 16.]
[+] Validation:
 [ 6.  5. 10.]


In [11]:
embeddings_index = {}
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print("[i] Completed!")

print("[i] Finished running setup.")

[i] Completed!
[i] Finished running setup.


In [12]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

def initial_boost(epoch):
    if epoch==0: return float(8.0)
    elif epoch==1: return float(4.0)
    elif epoch==2: return float(2.0)
    elif epoch==3: return float(1.5)
    else: return float(1.0)

def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.0001
        if epoch%33==0:multiplier = 10
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        #print("Epoch",epoch+1,"- learning_rate",rate)
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)

In [13]:
embedding_matrix_ns = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_ns[i] = embedding_vector
print("Completed!")

Completed!


In [14]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

In [15]:
# static channel
embedding_layer_frozen = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
embedded_sequences_frozen = embedding_layer_frozen(sequence_input)

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
# non-static channel
embedding_layer_train = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix_ns],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences_train = embedding_layer_train(sequence_input)

In [17]:
l_lstm1f = Bidirectional(LSTM(6,return_sequences=True,dropout=0.3, recurrent_dropout=0.0))(embedded_sequences_frozen)
l_lstm1t = Bidirectional(LSTM(6,return_sequences=True,dropout=0.3, recurrent_dropout=0.0))(embedded_sequences_train)
l_lstm1 = Concatenate(axis=1)([l_lstm1f, l_lstm1t])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
l_conv_2 = Conv1D(filters=24,kernel_size=2,activation='relu')(l_lstm1)
l_conv_2 = Dropout(0.3)(l_conv_2)
l_conv_3 = Conv1D(filters=24,kernel_size=3,activation='relu')(l_lstm1)
l_conv_3 = Dropout(0.3)(l_conv_3)

In [19]:
l_conv_5 = Conv1D(filters=24,kernel_size=5,activation='relu',)(l_lstm1)
l_conv_5 = Dropout(0.3)(l_conv_5)
l_conv_6 = Conv1D(filters=24,kernel_size=6,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(l_lstm1)
l_conv_6 = Dropout(0.3)(l_conv_6)

In [20]:
l_conv_8 = Conv1D(filters=24,kernel_size=8,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(l_lstm1)
l_conv_8 = Dropout(0.3)(l_conv_8)

In [21]:
conv_1 = [l_conv_6,l_conv_5, l_conv_8,l_conv_2,l_conv_3]

In [22]:
l_lstm_c = Concatenate(axis=1)(conv_1)

In [23]:
l_conv_4f = Conv1D(filters=12,kernel_size=4,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(embedded_sequences_frozen)
l_conv_4f = Dropout(0.3)(l_conv_4f)
l_conv_4t = Conv1D(filters=12,kernel_size=4,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(embedded_sequences_train)
l_conv_4t = Dropout(0.3)(l_conv_4t)

In [24]:
l_conv_3f = Conv1D(filters=12,kernel_size=3,activation='relu',)(embedded_sequences_frozen)
l_conv_3f = Dropout(0.3)(l_conv_3f)
l_conv_3t = Conv1D(filters=12,kernel_size=3,activation='relu',)(embedded_sequences_train)
l_conv_3t = Dropout(0.3)(l_conv_3t)

In [25]:
l_conv_2f = Conv1D(filters=12,kernel_size=2,activation='relu')(embedded_sequences_frozen)
l_conv_2f = Dropout(0.3)(l_conv_2f)
l_conv_2t = Conv1D(filters=12,kernel_size=2,activation='relu')(embedded_sequences_train)
l_conv_2t = Dropout(0.3)(l_conv_2t)

In [26]:
conv_2 = [l_conv_4f, l_conv_4t,l_conv_3f, l_conv_3t, l_conv_2f, l_conv_2t]

In [27]:
l_merge_2 = Concatenate(axis=1)(conv_2)
l_c_lstm = Bidirectional(LSTM(12,return_sequences=True,dropout=0.3, recurrent_dropout=0.0))(l_merge_2)

In [28]:
l_merge = Concatenate(axis=1)([l_lstm_c, l_c_lstm])
l_pool = MaxPooling1D(4)(l_merge)
l_drop = Dropout(0.5)(l_pool)
l_flat = Flatten()(l_drop)
l_dense = Dense(26, activation='relu')(l_flat)
preds = Dense(NUM_LABELS, activation='softmax')(l_dense)

In [29]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=0.9, rho=0.95, epsilon=None, decay=0.002)
lr_metric = get_lr_metric(adadelta)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [30]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint-0.91.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=0)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [31]:
model.summary()
model.save('BalanceNet.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 200)      19600       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 30, 200)      19600       input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 12)       9936        embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectio

In [32]:
print("Training Progress:")
model_log = model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=100, batch_size=128,
          callbacks=[tensorboard, model_checkpoints])

pd.DataFrame(model_log.history).to_csv("history-balance.csv")

Training Progress:
Instructions for updating:
Use tf.cast instead.
Train on 49 samples, validate on 21 samples
Epoch 1/100
49/49 [==============================] - 8s 155ms/step - loss: 1.1286 - acc: 0.4082 - val_loss: 1.0672 - val_acc: 0.4762
Epoch 2/100
49/49 [==============================] - 0s 4ms/step - loss: 1.1644 - acc: 0.4082 - val_loss: 1.1375 - val_acc: 0.2381
Epoch 3/100
49/49 [==============================] - 0s 6ms/step - loss: 1.1728 - acc: 0.3673 - val_loss: 1.0743 - val_acc: 0.4762
Epoch 4/100
49/49 [==============================] - 0s 5ms/step - loss: 1.1489 - acc: 0.2857 - val_loss: 1.1095 - val_acc: 0.2381
Epoch 5/100
49/49 [==============================] - 0s 6ms/step - loss: 1.0937 - acc: 0.3878 - val_loss: 1.0936 - val_acc: 0.4762
Epoch 6/100
49/49 [==============================] - 0s 5ms/step - loss: 1.1240 - acc: 0.3061 - val_loss: 1.1048 - val_acc: 0.2381
Epoch 7/100
49/49 [==============================] - 0s 5ms/step - loss: 1.1170 - acc: 0.3673 - val_l

Epoch 61/100
49/49 [==============================] - 0s 6ms/step - loss: 1.0978 - acc: 0.3469 - val_loss: 1.1004 - val_acc: 0.2381
Epoch 62/100
49/49 [==============================] - 0s 5ms/step - loss: 1.1000 - acc: 0.3469 - val_loss: 1.1079 - val_acc: 0.2381
Epoch 63/100
49/49 [==============================] - 0s 5ms/step - loss: 1.0934 - acc: 0.3673 - val_loss: 1.1169 - val_acc: 0.2381
Epoch 64/100
49/49 [==============================] - 0s 6ms/step - loss: 1.1185 - acc: 0.3878 - val_loss: 1.0960 - val_acc: 0.2381
Epoch 65/100
49/49 [==============================] - 0s 5ms/step - loss: 1.1052 - acc: 0.3061 - val_loss: 1.1060 - val_acc: 0.2381
Epoch 66/100
49/49 [==============================] - 0s 6ms/step - loss: 1.1042 - acc: 0.3061 - val_loss: 1.0995 - val_acc: 0.2857
Epoch 67/100
49/49 [==============================] - 0s 6ms/step - loss: 1.0860 - acc: 0.3469 - val_loss: 1.0951 - val_acc: 0.2381
Epoch 68/100
49/49 [==============================] - 0s 5ms/step - loss: 1.

In [33]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_format = 'retina'
import itertools, pickle

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

def classify(text):
    word = []
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    word.append(text)
    sequences_test = tokenizer.texts_to_sequences(word)
    data_int_t = pad_sequences(sequences_test, padding='pre', maxlen=(MAX_SEQUENCE_LENGTH-5))
    data_test = pad_sequences(data_int_t, padding='post', maxlen=(MAX_SEQUENCE_LENGTH))
    y_prob = model.predict(data_test)
    for n, prediction in enumerate(y_prob):
        pred = y_prob.argmax(axis=-1)[n]
        print(pred)

classify("수강신청 언제야")

1
